# NYBG DATA

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [5]:
filename_train = "/Users/annahauk/Desktop/bttai-nybg-2024/BTTAIxNYBG-train.csv"
df_train = pd.read_csv(filename_train, header = 0)

In [6]:
df_train

,uniqueID,classLabel,classID,source,imageFile
0,2,occluded-specimens,8,L,a1a8b48e8cb142b3.jpg
1,3,microscope-slides,6,L,79599db2ac9092b6.jpg
2,4,illustrations-color,2,BHL,c449696f2f0d0d92.jpg
3,5,illustrations-color,2,P,80a8f4a393b4e08c.jpg
4,6,animal-specimens,0,AK,041a1c6e73313638.jpg
...,...,...,...,...,...
81941,122874,micrographs-transmission-light,5,Tw,2424355d5438181c.jpg
81942,122875,illustrations-color,2,BHL,b28acccccecad04c.jpg
81943,122876,microscope-slides,6,L,78f6868694a6669c.jpg
81944,122877,live-plants,4,E,5c6162948949510a.jpg


## Provided Example Starting Notebook

In [ ]:
# Load dataset & Define image directory
df = pd.read_csv('/path/to/your/dataset.csv')  
# NOTE: if we have seperate train dataset and validate dataset, you may want to seperately load it with pd.read_csv() 
# Please update the paths when running this code
# train_df = pd.read_csv('/path/to/your/train_dataset.csv')
# validate_df = pd.read_csv('/path/to/your/validate_dataset.csv')


In [ ]:


image_directory = '/path/to/image/directory/'  # Please update this path when running this code

# Preprocessing function to load and process images
def load_and_preprocess_image(filename, target_size=(224, 224)):
    img_path = os.path.join(image_directory, filename)
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Model expects a batch of images
    return img_array / 255.0  # Normalize to [0, 1]

# Apply preprocessing to all images
df['imageData'] = df['imageFile'].apply(load_and_preprocess_image)

# Split dataset into training and validation sets
### Note: This is a common step in ML training, but in this challenge, since the validation set is provided separately, there is no need to call this function to distinguish between validation and train set.
# train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: No augmentation for validation data, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# Convert dataframe to a format suitable for the model training
def df_to_dataset(dataframe, datagen, batch_size=32):
    datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=image_directory,
        x_col='imageFile',
        y_col='classLabel',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Change this if not a multiclass classification
    )

# Create datasets for training and validation
train_dataset = df_to_dataset(train_df, train_datagen)
validation_dataset = df_to_dataset(validate_df, validation_datagen)

# This setup is now ready for training with model.fit using the train_dataset and validation_dataset
